In [77]:
from datetime import datetime

import time
import os
import sys
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.nddata import CCDData, Cutout2D
import astropy.units as u
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.table import Table
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

import ccdproc
import photutils

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

from mmtwfs.wfs import wfs_norm
from skycam_utils.photometry import make_background, make_segmentation_image, load_bright_star_catalog, make_catalog
from skycam_utils.pipeline import process_image
from skycam_utils.astrometry import solve_field

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%cd /Users/tim/MMT/sky_brightness/asi_skycam/Bias/

/Volumes/LaCie 8TB/sky_brightness/asi_skycam/Bias


In [ ]:
biases = ccdproc.ImageFileCollection('.', keywords="*", glob_include="Bias*")

In [ ]:
for h in biases.hdus(save_location='reduced', overwrite=True, ccd_kwargs={'unit': 'adu', 'gain': 1.0}):
    h.header['fixscale'] = True
    h.data = h.data / 16

In [ ]:
scaled_biases = ccdproc.ImageFileCollection('./reduced', keywords="*", glob_include="Bias_*")

In [ ]:
bias = ccdproc.combine(
    list(scaled_biases.ccds(ccd_kwargs={'unit': 'adu'})),
    method='average',
    mem_limit=1e9,
    clip_extreme=True,
    nlow=2,
    nhigh=2,
    dtype=np.float64
)
bias.write("./bias.fits", overwrite=True)

In [6]:
%cd ../Light

/Volumes/LaCie 8TB/sky_brightness/asi_skycam/Light


In [ ]:
filters = ['clear', 'u', 'g', 'r', 'i']
#filters = ['u']
for f in filters:
    rpath = f"{f}/reduced"
    if not Path.exists(Path(rpath)):
        Path.mkdir(Path(rpath))
    ic = ccdproc.ImageFileCollection(f"./{f}", keywords="*", glob_include="Light_*")
    for h in ic.hdus(save_location=rpath, overwrite=True, ccd_kwargs={'unit': u.adu, 'gain': 1.0}):
        h.header['fixscale'] = True
        h.data = h.data / 16
        h.data = h.data - bias.data

In [ ]:
test_im = CCDData.read(Path("./r/reduced/Light_024.fits"), unit=u.adu)
print(test_im.header['FILTER'])
Cutout2D(test_im, (300, 300), (100, 100), copy=True).data

In [15]:
fitsfile = Path("./g/reduced/Light_024.fits")

In [9]:
g_cat = process_image(fitsfile)

In [16]:
im = CCDData.read(fitsfile, unit=u.adu)

In [38]:
bkg = make_background(im)

In [39]:
bkg_image = CCDData(bkg.background, unit=u.adu)
bkg_image.write(fitsfile.with_suffix(".bkg.fits"), overwrite=True)

In [40]:
diff = CCDData(im.data - bkg_image.data, unit=u.adu)
diff_fp = fitsfile.with_suffix(".subt.fits")
diff.write(diff_fp, overwrite=True)

In [43]:
solved_fp = solve_field(diff_fp)

In [44]:
solved = CCDData.read(solved_fp)

In [45]:
segm = make_segmentation_image(solved)

In [70]:
s_cat, p_cat = make_catalog(solved, segm, solved.wcs)

In [71]:
s_cat

id,xcentroid,ycentroid,sky_centroid,sky_centroid_icrs,source_sum,source_sum_err,background_sum,background_mean,background_at_centroid,xmin,xmax,ymin,ymax,min_value,max_value,minval_xpos,minval_ypos,maxval_xpos,maxval_ypos,area,equivalent_radius,perimeter,semimajor_axis_sigma,semiminor_axis_sigma,eccentricity,orientation,ellipticity,elongation,covar_sigx2,covar_sigxy,covar_sigy2,cxx,cxy,cyy,obs_mag
,pix,pix,"deg,deg","deg,deg",,,,,,pix,pix,pix,pix,,,pix,pix,pix,pix,pix2,pix,pix,pix,pix,,rad,,,pix2,pix2,pix2,1 / pix2,1 / pix2,1 / pix2,
int64,float64,float64,object,object,float64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
79,398.4683022458725,239.2343571759362,"285.28093708055434,46.92316313080607","285.2809317692016,46.923157132260805",304.0278288559083,None,None,None,None,397.0,400.0,238.0,240.0,17.456186203683593,68.16248052811788,397.0,239.0,399.0,240.0,8.0,1.5957691216057308,8.242640687119284,1.0333257196735757,0.4806123555890215,0.8852517609727032,0.6695872652993615,0.5348878418115389,2.1500190489426103,0.7454184939355083,0.4072140407040448,0.5533317853482326,2.243475220442175,-3.302086140454444,3.022287828542472,-6.207283345250553
218,1191.07912285869,581.963241947118,"240.68687209062202,46.03195229342615","240.6868708200728,46.03194871099587",4625.161627036523,None,None,None,None,1187.0,1195.0,577.0,586.0,12.62231728848869,214.23324519225326,1187.0,584.0,1191.0,582.0,68.0,4.652426491681278,28.14213562373095,2.174237411278099,1.601902956697451,0.6761484335832841,-0.983452360437135,0.2632345720903625,1.3572840990072184,3.229768787566617,-0.9969356073429708,4.0636326157107066,0.3349871153509672,0.16436553934690704,0.266247230424965,-9.16281728458147
245,1026.8677052665507,651.3464757578001,"248.60329297267074,42.51587052164379","248.6032906183529,42.515866297050245",7657.345655480172,None,None,None,None,1022.0,1031.0,646.0,656.0,14.452655846063518,447.9549246499642,1024.0,647.0,1027.0,651.0,88.0,5.292567428401227,32.14213562373095,2.026969578353114,1.828513744616117,0.4315430420526512,-1.4657453860220435,0.0979076528115631,1.1085339578777196,3.35187538896576,-0.07978895325244781,4.100192796853295,0.2984787097000499,0.011616675115075265,0.24400399950501384,-9.710195629384145
250,584.7127189428252,655.2729073337395,"269.14345222381286,37.24376807173477","269.14344784704247,37.24376258236244",4818.7461546308905,None,None,None,None,578.0,589.0,651.0,659.0,4.253328451287027,305.2071967655674,589.0,655.0,586.0,656.0,83.0,5.140011726956917,31.556349186104043,2.5518869448718915,1.7695205223820605,0.7205369892061134,0.4335120805527364,0.30658349660122075,1.4421346984078147,5.915559893443587,1.2888195181295714,3.7277699650952885,0.18281635132806634,-0.12641192135299692,0.2901093913809559,-9.207335122068255
254,1221.242634493326,663.8990316220561,"238.62191660737247,43.12275760563621","238.62191496291064,43.12275420246672",457.44929208001315,None,None,None,None,1220.0,1223.0,662.0,666.0,18.534740221720764,48.57793451964358,1223.0,664.0,1222.0,664.0,14.0,2.111004122822376,11.071067811865476,1.1068453510264817,0.8631089128320462,0.6260389531557466,-1.1228303224293164,0.22020821424456083,1.2823936059178023,0.8350351357533122,-0.1874484326440806,1.1350284907457404,1.2436600100595465,0.4107775646664458,0.9149548348770701,-6.650857399610732
259,1229.6795089657585,682.558011960729,"238.09758088249532,42.45840932047295","238.0975791578943,42.45840596353454",3804.2293830307253,None,None,None,None,1226.0,1234.0,678.0,687.0,13.040777943117257,200.20312841673956,1233.0,685.0,1230.0,683.0,71.0,4.753945931439391,28.72792206135785,2.2233507303344044,1.709351217381155,0.6394681124647819,-0.8240958391284775,0.2311823797930168,1.300698597062301,3.854439349552967,-1.0076778763409433,4.010730704887999,0.277680214902017,0.1395317860226047,0.2668594891215832,-8.950666

In [72]:
filt = im.header['FILTER']
filt_col = f'{filt}_mag'
filt_col

'g_mag'

In [78]:
phot_off = p_cat[filt_col] - s_cat['obs_mag']
cut = p_cat[filt_col] < 4.0
zp = phot_off[cut].mean()
zp

13.833715874986774

In [80]:
pix_scales = proj_plane_pixel_scales(solved.wcs)
pix_area = pix_scales[0] * pix_scales[1] * 3600.**2
np.sqrt(pix_area)

138.649853421859

In [81]:
sky_mag = CCDData(zp + (-2.5 * np.log10(bkg_image.data/pix_area)), unit=u.adu)
sky_mag.write(fitsfile.with_suffix(".sky.fits"), overwrite=True)

In [95]:
mask = im.data <= 0
im.data[mask] = 1

/Users/tim/conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less_equal
  """Entry point for launching an IPython kernel.


In [96]:
orig_mag = CCDData(zp + (-2.5 * np.log10(im.data/pix_area)), unit=u.mag / u.arcsec**2)

In [97]:
orig_mag.write(fitsfile.with_suffix(".cal.fits"), overwrite=True)